In [1]:
import sys
sys.path.append("../")
import rg
from rg.graph import incidence_matrix
from rg.graph import ring_diagram ,show_spanning_trees
from rg.graph import circuits, set_circuit_flow,set_source_sink_flow,apply_flow
from rg.graph import scalar_propagator
#for inline dev
import numpy as np
from sympy import *
init_printing()

# TODO
 - Create the beautiful delta operator for graphs - this is at the level of incidence matrix and completely algebraic
 - Produce the final graph polynomials for the equilibrium graphs after the delta operator
 - We still have to integrate the graph polynomials in k
 - Some drawing mods and finalise then example of the simple mess from end to end
 - Gamma function - epsilon expansions

In [ ]:
class expr(object):
    def __init__(self, index = 0, sign=1, power=1, colour=1):
        self.value = 1/((sign*I*Symbol("omega_"+str(index)) + Symbol("C_"+str(colour))))**power
    
def _factors(a): return a.as_ordered_factors()

def _parts(f): 
    keys, values = zip(*f.as_powers_dict().items())
    exp = keys[0]
    value = f
    omega_term = [t for t in exp.free_symbols if "omega" in str(t)][0]
    sign = exp.coeff(I*omega_term)  
    return {"power": abs(values[0]), "exp": exp, "sign": sign, "omega": omega_term, "pole": solve(exp,omega_term)[0], "value" : f } 

def residue(G,P):
    r = G/P["value"]
    r = diff(r,P["omega"],n=P["power"]-1) if P["power"] > 1 else r
    r = r.subs({P["omega"]: P["pole"]})
    #plane*-1*I**(order) ) * ( (-1*I) **(order-1) )
    
    if P["power"] > 1:  r = r *  I**(P["power"]*1) *-1*I
    #if P["power"] > 1:  r =  I**(P["power"]) * (-1*I)**(P["power"]-1)
    r /= np.math.factorial(P["power"]-1)
    return r
def group_half_plane_residues(exp):
    terms = [_parts(f) for f in _factors(exp)]
    hterms = [t for t in terms if t["sign"]==-1] 
    d = {}
    for t in hterms:
        omega = t["omega"]
        if omega not in d:d[omega] = []
        d[omega].append(t)
    return d

def make_deltas_from_factors(exp):
    terms = [parts(f) for f in factors(exp)]
    d = {}
    for t in hterms:
        omega = t["omega"]
        if omega not in d:d[omega] = []
    for i, v in enumerate(terms): d[omega].append(v["sign"]*Symbol("alpha_"+str(i)))
        
    return d
    #get distinct loops
#groups = group_half_plane_residues(EG)

In [3]:
rainbow = incidence_matrix(4,4,
                       [ [0,1], [1,2], [2,3], [0,3],   ],
                       external_vertices=[0,2])
#apply_flow(rainbow,set_source_sink_flow)
ring_diagram(rainbow, options={"show_labels"})
#todo better drawing o

In [ ]:
maps = rainbow.edges_to_loop_basis()
L = rainbow.first_betti_number
species = ["a", "a", "b", "c"]
factor = rg._product_([scalar_propagator(maps[i][:L], maps[i][L:],species= species[i] ).value for i in maps.keys()])
factor
#todo make the species differ here

In [ ]:
#rainbow what are your delta functions
Matrix(rainbow._internal_edge_set_.X)

In [ ]:
list(rainbow._internal_edge_set_.X_delta)[0]
#actually this does not know about the double pole - what happens if we naievly apply?

In [ ]:
a,b=rainbow._symanzik_pols_(True,True,False,use_zero_mass=True)
a_,b_=rainbow._symanzik_pols_(True,True,True,use_zero_mass=True)
Matrix([simplify(a), simplify(a_),simplify(b),simplify(b_)])

In [ ]:
mat = [[0,1,0,0,1,],
       [0,1,0,1,1,],
       [0,-1,0,0,2,2]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG
B = IG.integrate()
B

In [ ]:
EG = expr(power=2).value*expr(sign=-1,colour=2).value#*expr(power=2).value
EG

In [ ]:
EG = expr(sign=-1,power=2).value*expr(sign=1,colour=2).value*expr(power=1,colour=3).value
EG

## what does the derivative look like

In [ ]:
F = expr(sign=1,colour=2).value*expr(power=1,colour=2).value
latex(diff(F, Symbol("omega_0"),n=1))

In [ ]:
expr(sign=1,colour=2).value*expr(power=1,colour=3).value

# What does this guys graph polynomial look like after sub

because this is one of the simplest diagrams that does not given a mono-rational

# integration

In [ ]:
groups = group_half_plane_residues(EG)
loops = list(groups.keys())
#for each loop perform an integration
T = rg._sum_([ residue(EG,P) for P in groups[loops[0]]])
latex(T)#simplify(T)

In [ ]:
apart(T,x=Symbol("C_1"))

# Schwinger integration

Here we produce the integration pre-factor which causes an expansion >= 1 and determines the weight of propagators in the propagator basis for each term

In [ ]:
make_deltas_from_factors(EG)

In [ ]:
loops = list(groups.keys())
loops[0]

In [ ]:
rg._sum_([ residue(EG,P) for P in groups[loops[0]]])

In [ ]:
EG = expr(power=2).value*expr(sign=-1,colour=2).value*expr(sign=-1,power=2,colour=3).value*expr(sign=1,power=1,colour=4).value
EG

In [ ]:
rg._sum_([ residue(EG,P) for P in groups[loops[0]]])

In [ ]:
EG = expr(power=2).value*\
expr(sign=-1,colour=2).value*\
expr(sign=-1,power=1,colour=3).value*\
expr(sign=1,power=1,colour=4).value
EG

In [ ]:
rg._sum_([ residue(EG,P) for P in groups[loops[0]]])

In [ ]:
sunmod = incidence_matrix(5,4,
                       [ [0,2], [0,3], [2,1], [3,1], [2,3] ],
                       external_vertices=[0,1])
apply_flow(sunmod,set_source_sink_flow)
ring_diagram(sunmod, options={"show_labels"})

In [ ]:
a,b=sunmod._symanzik_pols_(True,True,False)
expand(a)

In [ ]:
b

In [ ]:
show_spanning_trees(sunmod)

In [ ]:
merc = incidence_matrix(6,4,
                       [ [0,1], [1,2], [2,0], [0,3], [1,3], [2,3]  ],
                       external_vertices=[0,1,2])
apply_flow(merc,set_source_sink_flow)
ring_diagram(merc, options={"show_labels"})

In [ ]:
# IS = merc[:-1,merc.spanning_trees.iloc[0].values]
# np.matmul(np.linalg.pinv(IS) , IS )

In [ ]:
merc.constraint_system()

In [ ]:
merc.constraint_system(True)

In [ ]:
merc._internal_edge_set_.X

In [ ]:
list(merc._internal_edge_set_.X_delta)[0]

In [ ]:
list(merc._internal_edge_set_.X_delta)[1]

In [ ]:
list(merc._internal_edge_set_.X_delta)[2]

In [ ]:
len(list(merc._internal_edge_set_.X_delta))

In [ ]:
maps = merc.edges_to_loop_basis()
L = merc.first_betti_number
factor = rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])
factor

In [ ]:
IG = rg.integral(factor)
IG.integrate()

In [ ]:
mat = [[0,-1,0,0,1,],
       [0,1,0,0,0,1]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG

In [ ]:
#IG.integrate()

In [ ]:
basic_loop = incidence_matrix(2,2,
                       [ [0,1], [1,0],  ],
                       external_vertices=[0,1])
apply_flow(basic_loop,set_source_sink_flow)
ring_diagram(basic_loop, options={"show_labels"})

In [ ]:
basic_loop.constraint_system()

In [ ]:
basic_loop.constraint_system(True)

In [ ]:
sunset_boost = incidence_matrix(4,3,
                       [ [0,1], [1,0], [0,2], [2,1]  ],
                       external_vertices=[0,1])
apply_flow(sunset_boost,set_source_sink_flow)
ring_diagram(sunset_boost, options={"show_labels"})

In [ ]:
a,b=sunset_boost._symanzik_pols_(True,True,True)
a

In [ ]:
b

In [ ]:
ring_diagram(sunset_boost, options={"show_labels"})

In [ ]:
show_spanning_trees(sunset_boost)

In [ ]:
sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])
apply_flow(sunset,set_source_sink_flow)
ring_diagram(sunset, options={"show_labels"})

In [ ]:
sunset

## How to make an integral from an incidence matrix

In [ ]:
maps = sunset.edges_to_loop_basis()
L = sunset.first_betti_number
factor = rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])
factor

In [ ]:
IG = rg.integral(factor)
IG

In [ ]:
#IG.integrate([Symbol("omega_0")])
IG.integrate()


In [ ]:

k0,k1,kd1,D = symbols("k_0, k_1, k'_1 D")
exp = D*(kd1+k0+k1)**2
H = expand(exp)
H

In [ ]:
Poly(H)

In [ ]:
show_spanning_trees(sunset)

In [ ]:
sunset.constraint_system(True)

In [ ]:
a,b=_symanzik_pols_(sunset,True,True,False)
a

In [ ]:
b

In [ ]:
a,b=_symanzik_pols_(sunset,True,True,True)
a

In [ ]:
a,b=_symanzik_pols_(sunset,True,True,False)
a

In [ ]:
a,b=_symanzik_pols_(sunset,True,True,True)
a

In [ ]:
#now we need to fully integrate the sunset diagram to see what is going on 
#(later we can consider multiple poles and multiple species on other graphs)

# Tasks 
 - (a) make sure the integral formula is solid and then should work for any order
 - (b) some clean up on the K polynomials should be done but not a show stopper
 - (c) some enumerations for finding patterns at one then two loop and S species 

# Some one loop examples of omega-integration

# 1 
This is one of the simplest examples

In [ ]:
mat = [[0,-1,0,1,1,],
       [0,1,0,0,0,1]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG

In [ ]:
IG.integrate()

# 2 
example pole order 2 with externa momentum (freq)

In [ ]:
mat = [[0,1,0,1,0,1],
       [0,-1,0,0,0,2]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG

In [ ]:
IG.integrate()

# 3 

Example poles of higher order

In [ ]:
mat = [[0,-1,0,1,0,5],
       [0,1,0,0,0,2]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()


# 4 
Example introducing new species

In [ ]:
mat = [[0,-1,0,1,0,1],
       [0,1,0,1,1,1],
       [0,1,0,0,0,1]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG

In [ ]:
sunset.kirchhoff

In [ ]:
IG.integrate()

# Some two loop examples of omega-integration

In [ ]:
sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])
ring_diagram(sunset,options={"show_labels"})

In [ ]:
#if the loop integral has species colours, we have the option of the extended basis
#we can colour the edges too - go to rg.colour_map

In [ ]:
L = sunset.first_betti_number
maps = sunset.edges_to_loop_basis()
factor = rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])
IG = rg.integral(factor)
IG

In [ ]:
#this does not seem that believable unless the sunset is a special case for two-loop
IG.integrate()

In [ ]:
## more.... 

In [ ]:
bobcut = incidence_matrix(5,4,
                       [ [0,1], [1,3], [2,0], [3,2],[2,3] ],
                       external_vertices=[0,1])
ring_diagram(bobcut,options={"show_labels"})
#i know why the drawing is trying to create e1 which i can easily fix but how are the pids lining up?

In [ ]:
maps = bobcut.edges_to_loop_basis()
factor = rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])
IG = rg.integral(factor)
IG

In [ ]:
IG.integrate()

# Permute and integrate propagator combos
#TODO - should be able to test this by using the schwinger route and making sure they are in agreement

Then find a way to permute over propagators for S species, L loops, V external vertices - allow for vanishing integrals - create a table 

Look into leaving the fraction decomposition or finding it

In [ ]:
mat = [[0,-1,0,0,1,],
       [0,1,0,0,0,1]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()

In [ ]:
mat = [[0,-1,0,0,2,],
       [0,1,0,0,0,2]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()

In [ ]:
mat = [[0,-1,0,1,2,],
       [0,1,0,0,0,2]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()

In [ ]:
mat = [[0,-1,1,1,2,],
       [0,1,0,0,0,2]]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()

In [ ]:
mat = [[0,-1,1,1,2,],
       [1,1,0,0,0,2],
       [-1,1,0,0,0,2]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG.integrate()

In [ ]:
mat = [[0,1,1,1,1,],
       [0,1,0,0,0,1],
       [0,-1,0,0,1,2]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
IG#.integrate()

In [ ]:
#clearly the sum of two residues, maybe its better to do the fraction decomposition
#if this is possible from residues, how do we explain it with schwinger??? - remember the generliased schwinger rep
IG.integrate()

In [ ]:
#add some of Tauber and Freys examples for agreement check
#check for dealng with k momenta and then..
#move on then to doing the MS and eps expansion of the gamma functions - recheck with Cardy

In [ ]:
mat = [[0,1,1,0,1,],
       [0,-1,0,0,0,1]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
A = IG.integrate()
A

In [ ]:
mat = [[0,1,1,0,1,],
       [0,1,0,1,1,],
       [0,-1,0,0,0,1]
      ]
factors = scalar_propagator.from_basis_vector(mat,2)
IG = rg.integral(factors)
B = IG.integrate()
B

In [ ]:
simplify( (A.value+B.value).subs({Symbol("m_a"):0,Symbol("m_b"):0,Symbol("D_a"):Symbol("D"),Symbol("D_b"):Symbol("D")}))